# Biogeme Test - SMTO Data

Gonna try to adapt the Biogeme Swissmetro example to university choice using SMTO data set. We'll try to predict __mode__ choice using distance.

In [3]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable
import pandas as pd

hh_df = pd.read_csv('../../../Data/SMTO_2019/SMTO_2019_Households.csv', low_memory = False)
ps_df = pd.read_csv('../../../Data/SMTO_2019/SMTO_2019_Respondents.csv', low_memory = False)
zone_df = pd.read_csv('../../../Data/SMTO_2019/CampusZones.csv')

In [4]:
# Load relevant columns
df = ps_df[['pscampusmain']]
df = df.join(ps_df['psinstitution'])
df = df.join(hh_df[['HmTTS2006']])
df = df.join(ps_df[['psmainmodefalltypical']])
df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusmain': 'Campus', 'psmainmodefalltypical': 'Mode'})

campus_name_to_num = {}
keys = range(len(zone_df['Campus']))
values = list(zone_df['Campus'])
for i in keys:
    campus_name_to_num[values[i]] = i
#print(campus_name_to_num)


In [5]:
# Load relevant columns
df = ps_df[['pscampusmain']]
df = df.join(ps_df['psinstitution'])
df = df.join(hh_df[['HmTTS2006']])
df = df.join(ps_df[['psmainmodefalltypical']])
df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusmain': 'Campus', 'psmainmodefalltypical': 'Mode'})

# Convert Campus and Mode column to numerical column
# Create a Dictionary with Campus names from 0 to 26
campus_name_to_num = {}
keys = range(len(zone_df['Campus']))
values = list(zone_df['Campus'])
for i in keys:
    campus_name_to_num[values[i]] = i

mode_name_to_num = {"Drive alone": 0, "Drive with passenger(s) (household members only)": 0, "Drive with passenger(s) (including non-household members)": 0, "Auto passenger (driver is a household member)":0, "Auto passenger (driver is a non-household member)":0, "Ride-hailing alone (UberX, Lyft etc.)":0, "Ride-hailing with other passengers (Uberpool, Lyftpool etc.)": 0, "Taxi": 0, 
                    "Transit Bus": 1, "Streetcar": 1, "Subway/RT": 1, "GO Bus": 1, "GO Train": 1, 
                    "Walk": 2, "Bicycle": 2, "Bikeshare": 2}

df.loc[df['psinstitution'] == 'Ryerson University', 'Campus'] = 'Ryerson University'
df.loc[df['psinstitution'] == 'OCAD University', 'Campus'] = 'OCAD University'

df.replace({'Campus': campus_name_to_num}, inplace=True)
df.replace({'Mode': mode_name_to_num}, inplace=True)
print(df.shape)
#temp_df = df[df['psinstitution'] == 'Ryerson University']
#temp_df.head()

# Remove "Other" Modes
df = df[(df['Mode'] == 0) | (df['Mode'] == 1) | (df['Mode'] == 2)]
print(df.shape)
df['HomeZone'] = pd.to_numeric(df['HomeZone'], downcast='signed')
df['Mode'] = pd.to_numeric(df['Mode'], downcast='signed')
df['Campus'] = pd.to_numeric(df['Campus'], downcast='signed')
del df['psinstitution']

df = df.dropna() # Remove rows with missing data
print(df.shape)
df.head()

(19135, 4)
(10595, 4)
(10515, 3)


,Campus,HomeZone,Mode
0,2,3851.0,1
1,3,181.0,1
2,2,1039.0,1
5,2,544.0,1
11,2,548.0,1


In [8]:
# Dataframe with walk distances
df_path = pd.read_csv('../../../../LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

# Dataframe with AutoTravelTimes
df_att = pd.read_csv('../../../../LoS/Auto_Travel_Times.csv')
AutoTravelTimes = list(df_att['Data'])

# Dataframe with TransitTravelTimes
df_ttt = pd.read_csv('../../../../LoS/Transit_Travel_Times.csv')
TransitTravelTimes = list(df_ttt['Data'])

# Dataframe with TransitTravelTimes
df_ac = pd.read_csv('../../../../LoS/Auto_Cost.csv')
AutoCosts = list(df_ac['Data'])

# Dataframe with TransitTravelTimes
df_tc = pd.read_csv('../../../../LoS/Transit_Cost.csv')
TransitCosts = list(df_tc['Data'])

In [9]:
not_found = set()
# Function for distance/AutoTravelTime/TransitTravelTime lookup

def find_value(origin, destination, mode_num):
    # mode_num: 0 - to find Auto Travel Time
    #           1 - to find Transit Travel Time
    #           2 - to find Walking Distance
    #           3 - to find Auto Cost
    #           4 - to find Transit Cost
    try:
        i = origins.index(origin)
    except ValueError:
        not_found.add(origin)
        return 0
    try:
        j = origins.index(destination)
    except ValueError:
        not_found.add(destination)
        return 0
    
    if mode_num == 0:
        return AutoTravelTimes[i*2392 + j]
    elif mode_num == 1:
        return TransitTravelTimes[i*2392 + j]
    elif mode_num == 2:
        return dists[i*2392 + j] / 1000
    elif mode_num == 3:
        return AutoCosts[i*2392 + j]
    elif mode_num == 4:
        return TransitCosts[i*2392 + j]
    else:
        print("ERROR: Enter correct mode_num!")
        return 0

In [10]:
# List of campus' TTS zones from Joven's MOE data
campus_zones = list(zone_df['Zone'])
# Add column with campus zones
df['CampusZone'] = df['Campus'].apply(lambda x: campus_zones[x])

# Add columns for Distance, Transit Travel Time, Auto Travel Time
df['AIVTT'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 0), axis=1)
df['TPTT'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 1), axis=1)
df['Dist'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 2), axis=1)
df['AuCost'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 3), axis=1)
df['TrCost'] = df.apply(lambda x: find_value(x.HomeZone, x.CampusZone, 4), axis=1)

In [11]:
print(df.shape)
df.head()

(10515, 9)


,Campus,HomeZone,Mode,CampusZone,AIVTT,TPTT,Dist,AuCost,TrCost
0,2,3851.0,1,493,74.844180,140.742060,53.671230,3.943282,13.325980
1,3,181.0,1,564,20.281300,97.424699,21.717940,1.515612,2.220000
2,2,1039.0,1,493,24.659090,96.601530,17.852000,1.229752,6.822792
5,2,544.0,1,493,11.944880,56.045654,9.607668,0.706237,2.220000
11,2,548.0,1,493,8.111495,37.133500,6.225617,0.423913,2.220000


In [12]:
# Load data into Biogeme database
database = db.Database("SMTO", df)

# Definition of new variables: adding columns to the database 
Available =  DefineVariable('Available', 1, database) # All modes available to all students

# Make variable names global
globals().update(database.variables)

# Remove unknown values
database.remove(Dist == 0.0)
database.remove(TPTT == 0.0)
database.remove(AIVTT == 0.0)
database.remove(AIVTT > 0.1e20)

In [13]:
# Beta initialization: (name, value, lowerbound, upperbound, status, desc='')
# Status 0 if estimated, 1 if maintained - reference choice should be 1

ASC_AU = Beta('ASC_AU', 0, None, None, 1)
ASC_TR = Beta('ASC_TR', 0, None, None, 0)
ASC_AC = Beta('ASC_AC', 0, None, None, 0)
B_AIVTT = Beta('B_AIVTT', -0.05, None, None, 0)
B_TPTT = Beta('B_TPTT', -0.05, None, None, 0)
B_DIST = Beta('B_DIST', -0.0005, None, None, 0)
B_AUCOST = Beta('B_AUCOST', -0.0005, None, None, 0)
B_TRCOST = Beta('B_TRCOST', -0.0005, None, None, 0)

# Variables: from columns in database
AV = DefineVariable('AV', Available, database)
AU_AIVTT = DefineVariable('AU_AIVTT', AIVTT, database)
TR_TPTT = DefineVariable('TR_TPTT', TPTT, database)
AC_DIST = DefineVariable('AC_DIST', Dist, database)
AU_COST = DefineVariable('AU_COST', AuCost, database)
TR_COST = DefineVariable('TR_COST', TrCost, database)

# Mode Choice Utility Functions: ASC_AU is 0
V0 = ASC_AU + B_AIVTT * AU_AIVTT + B_AUCOST * AU_COST
V1 = ASC_TR + B_TPTT * TR_TPTT + B_TRCOST * TR_COST
V2 = ASC_AC + B_DIST * AC_DIST


V  = {0: V0, 1: V1, 2: V2}
av = {0: AV, 1: AV, 2: AV}

In [14]:
database.data

,Campus,HomeZone,Mode,CampusZone,AIVTT,TPTT,Dist,AuCost,TrCost,Available,AV,AU_AIVTT,TR_TPTT,AC_DIST,AU_COST,TR_COST
0,2,3851.0,1,493,74.844180,140.742060,53.671230,3.943282,13.325980,1,1.0,74.844180,140.742060,53.671230,3.943282,13.325980
1,3,181.0,1,564,20.281300,97.424699,21.717940,1.515612,2.220000,1,1.0,20.281300,97.424699,21.717940,1.515612,2.220000
2,2,1039.0,1,493,24.659090,96.601530,17.852000,1.229752,6.822792,1,1.0,24.659090,96.601530,17.852000,1.229752,6.822792
5,2,544.0,1,493,11.944880,56.045654,9.607668,0.706237,2.220000,1,1.0,11.944880,56.045654,9.607668,0.706237,2.220000
11,2,548.0,1,493,8.111495,37.133500,6.225617,0.423913,2.220000,1,1.0,8.111495,37.133500,6.225617,0.423913,2.220000
14,1,2611.0,1,526,80.103930,247.004720,53.223500,4.053303,13.421110,1,1.0,80.103930,247.004720,53.223500,4.053303,13.421110
19,3,615.0,1,564,7.260223,40.354967,5.677331,0.390787,2.220000,1,1.0,7.260223,40.354967,5.677331,0.390787,2.220000
21,0,249.0,1,282,13.301790,46.786401,5.633156,0.382239,2.220000,1,1.0,13.301790,46.786401,5.633156,0.382239,2.220000
23,2,4084.0,1,493,100.708900,341.983850,77.712020,5.422310,14.287340,1,1.0,100.708900,341.983850,77.712020,5.422310,14.287340
27,2,446.0,1,493,24.871310,83.488038,19.357610,1.399990,2.220000,1,1.0,24.871310,83.488038,19.357610,1.399990,2.220000


In [15]:
# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
logprob = models.loglogit(V, av, Mode)

# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDebug()
#logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()

# Create the Biogeme object
biogeme  = bio.BIOGEME(database,logprob,numberOfThreads=1)
biogeme.modelName = "SMTO_2019_ModeChoice_withCost"
#,numberOfThreads=1

# Estimate the parameters
results = biogeme.estimate(saveIterations=True)
biogeme.createLogFile()

# Print the estimated values
betas = results.getBetaValues()
for k,v in betas.items():
    print(f"{k:10}=\t{v:.3g}")

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[16:33:13] < General >   Remove 9 unused variables from the database as only 7 are used.
[16:33:14] < General >   Log likelihood (N=9667):   -81431.3
[16:33:14] < General >   Minimize with tol 1e-07
[16:33:14] < Debug >     ASC_AC:          0
[16:33:14] < Debug >     ASC_TR:          0
[16:33:14] < Debug >     B_AIVTT:      -0.05
[16:33:14] < Debug >     B_AUCOST:    -0.0005
[16:33:14] < Debug >     B_DIST:    -0.0005
[16:33:14] < Debug >     B_TPTT:      -0.05
[16:33:14] < Debug >     B_TRCOST:    -0.0005
[16:33:14] < General >   Log likelihood (N=9667):   -81431.3 Gradient norm:      2e+06  
[16:33:14] < Debug >     ASC_AC: -0.003538097
[16:33:14] < Debug >     ASC_TR: 0.003933501
[16:33:14] < Debug >     B_AIVTT: -0.02441806
[16:33:14] < Debug >     B_AUCOST: 0.002215146
[16:33:14] < Debug >     B_DIST: -0.1723948
[16:33:14] < Debug >     B_TPTT:  0.9344238
[16:33:14] < Debug >     B_TRCOST: 0.02542487
[16:33:14] < General >   Log likelihood (N=9667):  -971587.5 Gradient norm:      

[16:33:14] < Debug >     ASC_AC: -0.001171154
[16:33:14] < Debug >     ASC_TR: 0.001371733
[16:33:14] < Debug >     B_AIVTT: -0.03617529
[16:33:14] < Debug >     B_AUCOST: 0.0009245782
[16:33:14] < Debug >     B_DIST: -0.03253611
[16:33:14] < Debug >     B_TPTT: -0.002681934
[16:33:14] < Debug >     B_TRCOST: 0.005475805
[16:33:14] < General >   Log likelihood (N=9667):  -9394.252 Gradient norm:      6e+03  
[16:33:14] < Debug >     ASC_AC: -0.001383394
[16:33:14] < Debug >     ASC_TR: 0.001734115
[16:33:14] < Debug >     B_AIVTT: -0.03629905
[16:33:14] < Debug >     B_AUCOST: 0.000976428
[16:33:14] < Debug >     B_DIST: -0.0332715
[16:33:14] < Debug >     B_TPTT: -0.002721864
[16:33:14] < Debug >     B_TRCOST: 0.006520965
[16:33:14] < General >   Log likelihood (N=9667):  -9388.852 Gradient norm:      8e+03  
[16:33:14] < Debug >     ASC_AC: -0.002287175
[16:33:14] < Debug >     ASC_TR: 0.00330093
[16:33:14] < Debug >     B_AIVTT: -0.03736532
[16:33:14] < Debug >     B_AUCOST: 0.00115

[16:33:15] < Debug >     ASC_AC: -0.0536997
[16:33:15] < Debug >     ASC_TR:  0.1208618
[16:33:15] < Debug >     B_AIVTT: -0.01416403
[16:33:15] < Debug >     B_AUCOST: 0.04757097
[16:33:15] < Debug >     B_DIST: -0.006693273
[16:33:15] < Debug >     B_TPTT: -0.002309532
[16:33:15] < Debug >     B_TRCOST:  0.1960558
[16:33:15] < General >   Log likelihood (N=9667):  -8801.314 Gradient norm:      8e+04  
[16:33:15] < Debug >     ASC_AC: -0.07053098
[16:33:15] < Debug >     ASC_TR:  0.2068431
[16:33:15] < Debug >     B_AIVTT: -0.01964636
[16:33:15] < Debug >     B_AUCOST:  0.1172061
[16:33:15] < Debug >     B_DIST: -0.008756618
[16:33:15] < Debug >     B_TPTT: -0.002011537
[16:33:15] < Debug >     B_TRCOST:  0.1974855
[16:33:15] < General >   Log likelihood (N=9667):  -8832.182 Gradient norm:      3e+05  
[16:33:15] < Debug >     ASC_AC: -0.05910082
[16:33:15] < Debug >     ASC_TR:   0.148453
[16:33:15] < Debug >     B_AIVTT: -0.0159233
[16:33:15] < Debug >     B_AUCOST: 0.06991673
[16:3

[16:33:15] < Debug >     ASC_AC: -0.1210485
[16:33:15] < Debug >     ASC_TR:  0.5322144
[16:33:15] < Debug >     B_AIVTT:  0.0072143
[16:33:15] < Debug >     B_AUCOST:  0.0804094
[16:33:15] < Debug >     B_DIST: 0.01339725
[16:33:15] < Debug >     B_TPTT: -0.0009453694
[16:33:15] < Debug >     B_TRCOST:  0.2307966
[16:33:15] < General >   Log likelihood (N=9667):  -8625.714 Gradient norm:      5e+04  
[16:33:16] < Debug >     ASC_AC: -0.1180375
[16:33:16] < Debug >     ASC_TR:  0.4961932
[16:33:16] < Debug >     B_AIVTT: -0.001033763
[16:33:16] < Debug >     B_AUCOST:  0.1424949
[16:33:16] < Debug >     B_DIST: 0.01370531
[16:33:16] < Debug >     B_TPTT: -0.0007464347
[16:33:16] < Debug >     B_TRCOST:   0.220402
[16:33:16] < General >   Log likelihood (N=9667):  -8595.642 Gradient norm:      5e+04  
[16:33:16] < Debug >     ASC_AC: -0.1258185
[16:33:16] < Debug >     ASC_TR:  0.5666794
[16:33:16] < Debug >     B_AIVTT: 0.0002271193
[16:33:16] < Debug >     B_AUCOST:  0.1505746
[16:33:

[16:33:16] < Debug >     ASC_AC: -0.1514107
[16:33:16] < Debug >     ASC_TR:  0.9773039
[16:33:16] < Debug >     B_AIVTT: -0.01463691
[16:33:16] < Debug >     B_AUCOST: 0.06310868
[16:33:16] < Debug >     B_DIST: -0.006101851
[16:33:16] < Debug >     B_TPTT: -0.001229953
[16:33:16] < Debug >     B_TRCOST: 0.04647556
[16:33:16] < General >   Log likelihood (N=9667):  -8448.784 Gradient norm:      1e+03  
[16:33:16] < Debug >     ASC_AC: -0.1513925
[16:33:16] < Debug >     ASC_TR:  0.9780719
[16:33:16] < Debug >     B_AIVTT: -0.01467648
[16:33:16] < Debug >     B_AUCOST: 0.06300099
[16:33:16] < Debug >     B_DIST: -0.006151027
[16:33:16] < Debug >     B_TPTT: -0.001229329
[16:33:16] < Debug >     B_TRCOST: 0.04598897
[16:33:16] < General >   Log likelihood (N=9667):  -8448.779 Gradient norm:      1e+03  
[16:33:16] < Debug >     ASC_AC: -0.1512979
[16:33:16] < Debug >     ASC_TR:  0.9784574
[16:33:16] < Debug >     B_AIVTT: -0.01471341
[16:33:16] < Debug >     B_AUCOST: 0.06293587
[16:33

[16:33:17] < Debug >     ASC_AC: -0.1468622
[16:33:17] < Debug >     ASC_TR:  0.9808005
[16:33:17] < Debug >     B_AIVTT: -0.01541331
[16:33:17] < Debug >     B_AUCOST: 0.05708104
[16:33:17] < Debug >     B_DIST: -0.007291476
[16:33:17] < Debug >     B_TPTT: -0.001308918
[16:33:17] < Debug >     B_TRCOST: 0.04223207
[16:33:17] < General >   Log likelihood (N=9667):  -8448.232 Gradient norm:      2e+03  
[16:33:17] < Debug >     ASC_AC: -0.1432558
[16:33:17] < Debug >     ASC_TR:  0.9758189
[16:33:17] < Debug >     B_AIVTT: -0.0159596
[16:33:17] < Debug >     B_AUCOST: 0.05942692
[16:33:17] < Debug >     B_DIST: -0.007571826
[16:33:17] < Debug >     B_TPTT: -0.001336859
[16:33:17] < Debug >     B_TRCOST: 0.04211243
[16:33:17] < General >   Log likelihood (N=9667):  -8447.826 Gradient norm:      2e+03  
[16:33:17] < Debug >     ASC_AC: -0.1347351
[16:33:17] < Debug >     ASC_TR:  0.9679809
[16:33:17] < Debug >     B_AIVTT: -0.01654614
[16:33:17] < Debug >     B_AUCOST: 0.06078057
[16:33:

[16:33:17] < Debug >     ASC_AC:  0.3421146
[16:33:17] < Debug >     ASC_TR:   1.280633
[16:33:17] < Debug >     B_AIVTT: -0.007109962
[16:33:17] < Debug >     B_AUCOST: 0.0007466162
[16:33:17] < Debug >     B_DIST: -0.008309033
[16:33:17] < Debug >     B_TPTT: -0.001310208
[16:33:17] < Debug >     B_TRCOST: 0.04438017
[16:33:17] < General >   Log likelihood (N=9667):  -8384.162 Gradient norm:      2e+03  
[16:33:17] < Debug >     ASC_AC:  0.3382802
[16:33:17] < Debug >     ASC_TR:   1.271028
[16:33:17] < Debug >     B_AIVTT: -0.007522039
[16:33:17] < Debug >     B_AUCOST: 0.01206556
[16:33:17] < Debug >     B_DIST: -0.007888477
[16:33:17] < Debug >     B_TPTT: -0.001287217
[16:33:17] < Debug >     B_TRCOST: 0.04893914
[16:33:17] < General >   Log likelihood (N=9667):  -8382.301 Gradient norm:      1e+04  
[16:33:17] < Debug >     ASC_AC:  0.3596369
[16:33:17] < Debug >     ASC_TR:   1.276741
[16:33:17] < Debug >     B_AIVTT: -0.006797211
[16:33:17] < Debug >     B_AUCOST: 0.02377259
[

[16:33:18] < Debug >     ASC_AC:  0.4360067
[16:33:18] < Debug >     ASC_TR:    1.32579
[16:33:18] < Debug >     B_AIVTT: -0.001963956
[16:33:18] < Debug >     B_AUCOST: 0.03344836
[16:33:18] < Debug >     B_DIST: -0.001876084
[16:33:18] < Debug >     B_TPTT: -0.000966081
[16:33:18] < Debug >     B_TRCOST: 0.07867186
[16:33:18] < General >   Log likelihood (N=9667):  -8374.049 Gradient norm:      1e+03  
[16:33:18] < Debug >     ASC_AC:  0.4359329
[16:33:18] < Debug >     ASC_TR:    1.32649
[16:33:18] < Debug >     B_AIVTT: -0.001947935
[16:33:18] < Debug >     B_AUCOST:  0.0332106
[16:33:18] < Debug >     B_DIST: -0.001887956
[16:33:18] < Debug >     B_TPTT: -0.0009644983
[16:33:18] < Debug >     B_TRCOST:  0.0785078
[16:33:18] < General >   Log likelihood (N=9667):  -8374.048 Gradient norm:      4e+01  
[16:33:18] < Debug >     ASC_AC:  0.4362931
[16:33:18] < Debug >     ASC_TR:   1.326733
[16:33:18] < Debug >     B_AIVTT: -0.00191886
[16:33:18] < Debug >     B_AUCOST: 0.03298931
[16

[16:33:19] < Debug >     ASC_AC:  0.4365785
[16:33:19] < Debug >     ASC_TR:   1.326898
[16:33:19] < Debug >     B_AIVTT: -0.001906106
[16:33:19] < Debug >     B_AUCOST: 0.03288373
[16:33:19] < Debug >     B_DIST: -0.001881005
[16:33:19] < Debug >     B_TPTT: -0.000965735
[16:33:19] < Debug >     B_TRCOST: 0.07866819
[16:33:19] < General >   Log likelihood (N=9667):  -8374.047 Gradient norm:        0.8  
[16:33:19] < Debug >     ASC_AC:  0.4365805
[16:33:19] < Debug >     ASC_TR:   1.326899
[16:33:19] < Debug >     B_AIVTT: -0.001906115
[16:33:19] < Debug >     B_AUCOST: 0.03288382
[16:33:19] < Debug >     B_DIST: -0.001881013
[16:33:19] < Debug >     B_TPTT: -0.0009657355
[16:33:19] < Debug >     B_TRCOST: 0.07866821
[16:33:19] < General >   Log likelihood (N=9667):  -8374.047 Gradient norm:        0.8  
[16:33:19] < Debug >     ASC_AC:  0.4365957
[16:33:19] < Debug >     ASC_TR:   1.326897
[16:33:19] < Debug >     B_AIVTT: -0.001905612
[16:33:19] < Debug >     B_AUCOST: 0.03289104
[1

Upon running, this produces several output files. The most important is the html file.

### Resources
UT Austin example: https://www.youtube.com/watch?v=QeJgyBIaXMQ  
Biogeme example: https://www.youtube.com/watch?v=OiM94B8WayA  
Nested logit example: https://www.youtube.com/watch?v=vEhvf54IKvs